In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
# remove pandas colomn limit
pd.set_option('display.max_columns', None)


* Im going to create one data frame with all the points and features and then I will cluster them for each segement using a time series aproch

* then i will use those cluster lables for the second data frame where it have a row for each segment , with speed acceleration and other features and their mean , median , MAX , MIN and std

* and i will use that second dataframe to do a clustering based on driving behavior , 

* then i will try to find a meaning those clusters have. then for the training dataset i have to train a clasifire that will predict the cluster of each future segment

* then using those data i will pridict the arrival time.

* if that was not enough i will train a regression model to predict the speed , acceleration and other features for each new upcoming segemnt, and then i will use that to predict the arrival time


In [2]:
# import csv DataOut\bus_running_times_feature_added_all_moddedDF.csv
running_time_features = pd.read_csv('../DataOut/bus_running_times_feature_added_all_moddedDF.csv')
# read csv DataOut\bus_trip_all_points_moddedDF.csv 
bus_trip_speed_and_other_features = pd.read_csv('../DataOut/bus_trip_all_points_with_acceleration_and_radial_acceleration_and_distance_and_accDiff.csv')

In [3]:
# select a random set of 10 trip ids , sed a known seed for reproducibility
np.random.seed(42)
trip_ids = np.random.choice(bus_trip_speed_and_other_features.trip_id.unique(), 10, replace=False)
# select only the trips with the selected trip ids
bus_trip_speed_and_other_features = bus_trip_speed_and_other_features[bus_trip_speed_and_other_features.trip_id.isin(trip_ids)]
running_time_features= running_time_features[running_time_features.trip_id.isin(trip_ids)]
# reset the index
bus_trip_speed_and_other_features.reset_index(drop=True, inplace=True)
running_time_features.reset_index(drop=True, inplace=True)

trip_ids

array([4307., 1266., 1189.,  476., 2918., 3048., 3562., 3263., 4519.,
       5228.])

In [4]:
running_time_features=running_time_features[running_time_features['trip_id'].isin(trip_ids)]
bus_trip_speed_and_other_features=bus_trip_speed_and_other_features[bus_trip_speed_and_other_features['trip_id'].isin(trip_ids)]

In [5]:
useless_features=['id','geometry','count',]
bus_trip_speed_and_other_features=bus_trip_speed_and_other_features.drop(useless_features,axis=1)

In [6]:
bus_trip_speed_and_other_features

,deviceid,devicetime,latitude,longitude,speed,date,time,bus_stop,trip_id,direction,acceleration,radial_acceleration,distance_from_start,time_diff,acc_diff,acceleration_der
0,116,2022-01-21 07:33:16,7.293090,80.635632,0.00000,2022-01-21,07:33:16,NaN,476.0,1,-0.359971,0.000000,0.000000,NaN,NaN,0.000000
1,116,2022-01-21 07:33:31,7.293090,80.635872,7.01944,2022-01-21,07:33:31,NaN,476.0,1,0.467963,0.068102,26.470880,0 days 00:00:15,0.827934,0.055196
2,116,2022-01-21 07:33:46,7.293035,80.636608,11.33910,2022-01-21,07:33:46,NaN,476.0,1,0.287977,0.033749,107.955285,0 days 00:00:15,-0.179985,-0.011999
3,116,2022-01-21 07:34:01,7.292993,80.637405,9.71923,2022-01-21,07:34:01,NaN,476.0,1,-0.107991,0.264496,195.949844,0 days 00:00:15,-0.395969,-0.026398
4,116,2022-01-21 07:34:16,7.292930,80.637688,0.00000,2022-01-21,07:34:16,NaN,476.0,1,-0.647949,0.000000,227.981921,0 days 00:00:15,-0.539957,-0.035997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,262,2021-12-18 13:46:04,7.290942,80.637880,12.41900,2021-12-18,13:46:04,NaN,5228.0,2,0.647948,0.306767,15476.526207,0 days 00:00:15,0.107992,0.007199
2306,262,2021-12-18 13:46:19,7.291202,80.637305,9.71923,2021-12-18,13:46:19,NaN,5228.0,2,-0.179985,0.338331,15546.225109,0 days 00:00:15,-0.827933,-0.055196
2307,262,2021-12-18 13:46:34,7.291315,80.636527,10.25920,2021-12-18,13:46:34,NaN,5228.0,2,0.035998,0.087290,15632.998777,0 days 00:00:15,0.215983,0.014399
2308,262,2021-12-18 13:46:49,7.291537,80.635562,11.87910,2021-12-18,13:46:49,NaN,5228.0,2,0.107993,0.037240,15742.251739,0 days 00:00:15,0.071995,0.004800


In [7]:
running_time_features

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds,length,day_of_week,time_of_day,Sunday/holiday,saturday,weekday/end,week_no,rt(w-1),rt(w-2),rt(w-3),rt(t-1),rt(t-2),rt(n-1),rt(n-2),rt(n-3),hour_of_day,day,month,temp,precip,windspeed,conditions,dt(n-1)
0,3562.0,128.0,1.0,1.0,2021-10-29,15:01:42,15:03:36,0 days 00:01:54,114.0,0.6261,4.0,15.00,0.0,0,1,5.0,106.0,106.0,106.0,90.0,136.0,106.0,106.0,106.0,15.0,29.0,10.0,24.9,0.0,6.8,Partially cloudy,0.0
1,3562.0,128.0,1.0,2.0,2021-10-29,15:05:44,15:12:04,0 days 00:06:20,380.0,1.2808,4.0,15.00,0.0,0,1,5.0,238.0,238.0,238.0,238.0,238.0,114.0,238.0,238.0,15.0,29.0,10.0,24.9,0.0,6.8,Partially cloudy,128.0
2,3562.0,128.0,1.0,3.0,2021-10-29,15:12:24,15:18:13,0 days 00:05:49,349.0,2.1125,4.0,15.00,0.0,0,1,5.0,437.0,437.0,347.0,437.0,437.0,380.0,114.0,437.0,15.0,29.0,10.0,24.9,0.0,6.8,Partially cloudy,20.0
3,3562.0,128.0,1.0,4.0,2021-10-29,15:18:13,15:21:43,0 days 00:03:30,210.0,1.5513,4.0,15.25,0.0,0,1,5.0,233.0,233.0,233.0,260.0,280.0,349.0,380.0,114.0,15.0,29.0,10.0,24.9,0.0,6.8,Partially cloudy,0.0
4,3562.0,128.0,1.0,5.0,2021-10-29,15:21:43,15:23:31,0 days 00:01:48,108.0,0.8450,4.0,15.25,0.0,0,1,5.0,137.0,137.0,137.0,137.0,137.0,210.0,349.0,380.0,15.0,29.0,10.0,24.9,0.0,6.8,Partially cloudy,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,3263.0,123.0,1.0,11.0,2022-09-15,17:13:20,17:16:13,0:02:53,173.0,1.1000,3.0,17.00,0.0,0,1,35.0,234.0,151.0,168.0,244.0,193.0,158.0,34.0,60.0,17.0,15.0,9.0,24.2,0.0,6.1,Partially cloudy,15.0
142,3263.0,123.0,1.0,12.0,2022-09-15,17:16:28,17:19:03,0:02:35,155.0,1.3100,3.0,17.25,0.0,0,1,35.0,183.0,183.0,183.0,183.0,183.0,173.0,158.0,34.0,17.0,15.0,9.0,24.2,0.0,6.1,Partially cloudy,15.0
143,3263.0,123.0,1.0,13.0,2022-09-15,17:19:17,17:21:18,0:02:01,121.0,1.1500,3.0,17.25,0.0,0,1,35.0,150.0,150.0,150.0,150.0,158.0,155.0,173.0,158.0,17.0,15.0,9.0,24.2,0.0,6.1,Partially cloudy,14.0
144,3263.0,123.0,1.0,14.0,2022-09-15,17:21:33,17:29:40,0:08:07,487.0,0.8900,3.0,17.25,0.0,0,1,35.0,98.0,141.0,160.0,440.0,124.0,121.0,155.0,173.0,17.0,15.0,9.0,24.2,0.0,6.1,Partially cloudy,15.0


In [8]:
#print unique set of trip ids where direction is equal to 2
print(running_time_features[running_time_features['direction']==2]['trip_id'].unique())

[4307. 5228. 4519. 3048.]


Insert segment column 

In [9]:
# set Nan values in bus_stop column to 0
bus_trip_speed_and_other_features['bus_stop']=bus_trip_speed_and_other_features['bus_stop'].fillna(0)

In [10]:
ayyas_df = pd.read_csv('../DataOut/bus_running_times_feature_added_all_moddedDF.csv')

In [11]:

bus_trip_speed_and_other_features['segment'] = 0
for x in bus_trip_speed_and_other_features.trip_id.unique():
    df = bus_trip_speed_and_other_features[bus_trip_speed_and_other_features['trip_id'] == x]
    ayyas_tid_df = ayyas_df[ayyas_df['trip_id'] == x]

    for index, row in df.iterrows():
        for index2, row2 in ayyas_tid_df.iterrows():
            # print(row)
            print_counter = 0
            # Check if 'start_time' and 'end_time' are not null (not missing values)
            if pd.notna(row2['start_time']) and pd.notna(row2['end_time']):
                if row2['start_time'] <= row['time'] and row['time'] <= row2['end_time']:
                    bus_trip_speed_and_other_features.loc[index, 'segment'] = row2['segment']
                    break
                        

In [12]:
bus_trip_speed_and_other_features

,deviceid,devicetime,latitude,longitude,speed,date,time,bus_stop,trip_id,direction,acceleration,radial_acceleration,distance_from_start,time_diff,acc_diff,acceleration_der,segment
0,116,2022-01-21 07:33:16,7.293090,80.635632,0.00000,2022-01-21,07:33:16,0,476.0,1,-0.359971,0.000000,0.000000,NaN,NaN,0.000000,0
1,116,2022-01-21 07:33:31,7.293090,80.635872,7.01944,2022-01-21,07:33:31,0,476.0,1,0.467963,0.068102,26.470880,0 days 00:00:15,0.827934,0.055196,1
2,116,2022-01-21 07:33:46,7.293035,80.636608,11.33910,2022-01-21,07:33:46,0,476.0,1,0.287977,0.033749,107.955285,0 days 00:00:15,-0.179985,-0.011999,1
3,116,2022-01-21 07:34:01,7.292993,80.637405,9.71923,2022-01-21,07:34:01,0,476.0,1,-0.107991,0.264496,195.949844,0 days 00:00:15,-0.395969,-0.026398,1
4,116,2022-01-21 07:34:16,7.292930,80.637688,0.00000,2022-01-21,07:34:16,0,476.0,1,-0.647949,0.000000,227.981921,0 days 00:00:15,-0.539957,-0.035997,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,262,2021-12-18 13:46:04,7.290942,80.637880,12.41900,2021-12-18,13:46:04,0,5228.0,2,0.647948,0.306767,15476.526207,0 days 00:00:15,0.107992,0.007199,34
2306,262,2021-12-18 13:46:19,7.291202,80.637305,9.71923,2021-12-18,13:46:19,0,5228.0,2,-0.179985,0.338331,15546.225109,0 days 00:00:15,-0.827933,-0.055196,34
2307,262,2021-12-18 13:46:34,7.291315,80.636527,10.25920,2021-12-18,13:46:34,0,5228.0,2,0.035998,0.087290,15632.998777,0 days 00:00:15,0.215983,0.014399,34
2308,262,2021-12-18 13:46:49,7.291537,80.635562,11.87910,2021-12-18,13:46:49,0,5228.0,2,0.107993,0.037240,15742.251739,0 days 00:00:15,0.071995,0.004800,34


# TS cluster segemnts

In [13]:
df_for_seg_ts_clustering = bus_trip_speed_and_other_features[['trip_id','segment','time','distance_from_start','speed','acceleration','radial_acceleration','acceleration_der']]
# drop rows where segment is 0
df_for_seg_ts_clustering = df_for_seg_ts_clustering[df_for_seg_ts_clustering['segment'] != 0]
df_for_seg_ts_clustering

,trip_id,segment,time,distance_from_start,speed,acceleration,radial_acceleration,acceleration_der
1,476.0,1,07:33:31,26.470880,7.01944,0.467963,0.068102,0.055196
2,476.0,1,07:33:46,107.955285,11.33910,0.287977,0.033749,-0.011999
3,476.0,1,07:34:01,195.949844,9.71923,-0.107991,0.264496,-0.026398
4,476.0,1,07:34:16,227.981921,0.00000,-0.647949,0.000000,-0.035997
5,476.0,1,07:34:31,277.806275,7.55940,0.503960,0.177178,0.076794
...,...,...,...,...,...,...,...,...
2304,5228.0,34,13:45:49,15400.624372,2.69978,0.539956,0.066360,0.323973
2305,5228.0,34,13:46:04,15476.526207,12.41900,0.647948,0.306767,0.007199
2306,5228.0,34,13:46:19,15546.225109,9.71923,-0.179985,0.338331,-0.055196
2307,5228.0,34,13:46:34,15632.998777,10.25920,0.035998,0.087290,0.014399


In [14]:
import pandas as pd

# Assuming df_for_seg_ts_clustering is your DataFrame

# Convert the 'time' column to datetime objects
df_for_seg_ts_clustering['time'] = pd.to_datetime(df_for_seg_ts_clustering['time'], format='%H:%M:%S')

# Group by 'trip_id' and 'segment'
grouped = df_for_seg_ts_clustering.groupby(['trip_id', 'segment'])

# Define a function to calculate the time difference in seconds within each group
def calculate_time_diff(group):
    group['time'] = (group['time'] - group['time'].min()).dt.total_seconds().astype(int)
    return group

# Apply the function to each group and concatenate the results
df_for_seg_ts_clustering = grouped.apply(calculate_time_diff)


In [24]:
# Print the resulting DataFrame
df_for_seg_ts_clustering[df_for_seg_ts_clustering['trip_id'] == 476].head(50)

,trip_id,segment,time,distance_from_start,speed,acceleration,radial_acceleration,acceleration_der
0,476.0,1,0,26.470880,7.01944,0.467963,0.068102,0.055196
1,476.0,1,15,107.955285,11.33910,0.287977,0.033749,-0.011999
2,476.0,1,30,195.949844,9.71923,-0.107991,0.264496,-0.026398
3,476.0,1,45,227.981921,0.00000,-0.647949,0.000000,-0.035997
4,476.0,1,60,277.806275,7.55940,0.503960,0.177178,0.076794
5,476.0,1,75,354.258956,8.63931,0.071994,0.155917,-0.028798
6,476.0,1,90,402.029744,0.00000,-0.575954,0.000000,-0.043197
7,476.0,2,0,514.573434,9.71923,0.647949,0.291465,0.059995
8,476.0,2,15,601.448165,10.79910,0.071991,0.378892,-0.038397
9,476.0,2,30,681.541615,10.79910,0.000000,0.684094,-0.004799


In [16]:
df_for_seg_ts_clustering=df_for_seg_ts_clustering.drop(['trip_id','segment'],axis=1)
df_for_seg_ts_clustering=df_for_seg_ts_clustering.reset_index()
# drop level_2 column
df_for_seg_ts_clustering = df_for_seg_ts_clustering.drop('level_2',axis=1)


In [17]:
# Group by 'segment' and calculate the maximum time in seconds for each segment within each trip_id
grouped = df_for_seg_ts_clustering.groupby(['segment', 'trip_id'])['time'].max()

# Calculate the average total time length for each segment
segment_avg_times = grouped.groupby('segment').mean()

# Round the mean value to the nearest 15th multiplier integer
segment_avg_times_rounded = (segment_avg_times / 15).round() * 15 

# Convert the result to a dictionary
segment_avg_times_dict = segment_avg_times_rounded.to_dict()

# Print the dictionary
print(segment_avg_times_dict)


{1: 105.0, 2: 240.0, 3: 465.0, 4: 225.0, 5: 120.0, 6: 225.0, 7: 45.0, 8: 75.0, 9: 30.0, 10: 180.0, 11: 180.0, 12: 150.0, 13: 150.0, 14: 90.0, 15: 285.0, 21: 375.0, 22: 105.0, 23: 210.0, 24: 180.0, 25: 240.0, 26: 135.0, 27: 45.0, 28: 150.0, 29: 195.0, 30: 420.0, 31: 45.0, 32: 255.0, 33: 225.0, 34: 75.0}


In [22]:
import pandas as pd
import numpy as np

# Assuming df_for_seg_ts_clustering is your DataFrame with the time adjustments
# Assuming segment_avg_times_dict is the dictionary of rounded average times you calculated earlier

# Create a new DataFrame to store the adjusted data
adjusted_df = pd.DataFrame(columns=df_for_seg_ts_clustering.columns)

# Iterate through each trip ID
for trip_id in df_for_seg_ts_clustering['trip_id'].unique():
    # Get the unique segments for this trip ID
    segments = df_for_seg_ts_clustering[df_for_seg_ts_clustering['trip_id'] == trip_id]['segment'].unique()
    
    # Iterate through each segment
    for segment in segments:
        # Get the data for this trip ID and segment
        segment_data = df_for_seg_ts_clustering[(df_for_seg_ts_clustering['trip_id'] == trip_id) & (df_for_seg_ts_clustering['segment'] == segment)]
        
        # Get the expected maximum time based on the rounded average time for this segment
        expected_max_time = segment_avg_times_dict.get(segment, 0)
        
        # Check if the segment_data needs adjustment
        if len(segment_data) > 0:
            min_time = segment_data['time'].min()
            max_time = segment_data['time'].max()
            
            # Check if the segment data needs adjustment
            if (min_time == 0 and max_time == expected_max_time):
                # Data is already as expected, concatenate it to the adjusted DataFrame
                adjusted_df = pd.concat([adjusted_df, segment_data])
            else:
                # Data needs adjustment, create a sequence of 15-second intervals
                expected_times = np.arange(0, expected_max_time + 15, 15)
                
                # Create a list of dictionaries for the new rows with NaN values
                new_rows = []
                for expected_time in expected_times:
                    if expected_time not in segment_data['time'].values:
                        new_row = {'trip_id': trip_id, 'segment': segment, 'time': expected_time}
                        new_rows.append(new_row)
                
                # Create a DataFrame from the list of dictionaries
                new_rows_df = pd.DataFrame(new_rows, columns=adjusted_df.columns)
                
                # Concatenate the new rows DataFrame with the adjusted DataFrame
                adjusted_df = pd.concat([adjusted_df, new_rows_df], ignore_index=True)
        else:
            # No data for this trip ID and segment, insert rows with NaN values
            expected_times = np.arange(0, expected_max_time + 15, 15)
            
            # Create a list of dictionaries for the new rows with NaN values
            new_rows = []
            for expected_time in expected_times:
                new_row = {'trip_id': trip_id, 'segment': segment, 'time': expected_time}
                new_rows.append(new_row)
            
            # Create a DataFrame from the list of dictionaries
            new_rows_df = pd.DataFrame(new_rows, columns=adjusted_df.columns)
            
            # Concatenate the new rows DataFrame with the adjusted DataFrame
            adjusted_df = pd.concat([adjusted_df, new_rows_df], ignore_index=True)

# Sort the adjusted DataFrame
adjusted_df.sort_values(by=['trip_id', 'segment', 'time'], inplace=True)

# Reset the index
adjusted_df.reset_index(drop=True, inplace=True)

# Print the adjusted DataFrame
print(adjusted_df)


      trip_id segment   time  distance_from_start     speed  acceleration  \
0       476.0       1  105.0                  NaN       NaN           NaN   
1       476.0       2    0.0           514.573434   9.71923      0.647949   
2       476.0       2   15.0           601.448165  10.79910      0.071991   
3       476.0       2   30.0           681.541615  10.79910      0.000000   
4       476.0       2   45.0           749.130042  12.95900      0.143993   
...       ...     ...    ...                  ...       ...           ...   
1217   5228.0      34   15.0                  NaN       NaN           NaN   
1218   5228.0      34   30.0                  NaN       NaN           NaN   
1219   5228.0      34   45.0                  NaN       NaN           NaN   
1220   5228.0      34   60.0                  NaN       NaN           NaN   
1221   5228.0      34   75.0                  NaN       NaN           NaN   

      radial_acceleration  acceleration_der  
0                     NaN    

In [23]:
adjusted_df.head(50)

,trip_id,segment,time,distance_from_start,speed,acceleration,radial_acceleration,acceleration_der
0,476.0,1,105.0,NaN,NaN,NaN,NaN,NaN
1,476.0,2,0.0,514.573434,9.71923,0.647949,0.291465,0.059995
2,476.0,2,15.0,601.448165,10.79910,0.071991,0.378892,-0.038397
3,476.0,2,30.0,681.541615,10.79910,0.000000,0.684094,-0.004799
4,476.0,2,45.0,749.130042,12.95900,0.143993,0.455704,0.009600
5,476.0,2,60.0,838.471877,7.01944,-0.395971,0.560575,-0.035998
6,476.0,2,75.0,894.327403,8.09935,0.071994,0.016037,0.031198
7,476.0,2,90.0,967.251675,6.47948,-0.107991,0.204732,-0.011999
8,476.0,2,105.0,1022.403368,3.23974,-0.215983,0.196898,-0.007199
9,476.0,2,120.0,1037.657317,3.77970,0.035997,0.014704,0.016799
